# Overview

This Notebook demonstrates how to interact with the test app API.  Remember that you can see the documented endpoints at http://localhost:8000/docs.  Note that the `base_url` in our client says `backend:8000`, which is the right domain for code in the Jupyter container to communicate to the test app in the same docker network.  Your browser is not in that docker network, which is why it's navigating to `localhost:8000`.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import httpx

client = httpx.Client(base_url="http://backend:8000", follow_redirects=True)
client

<IPython.core.display.Javascript object>

# Login

Before we can do anything with the Todo endpoints we need to log in

In [3]:
resp = client.get("/me")
resp

<Response [401 Unauthorized]>

<IPython.core.display.Javascript object>

In [4]:
resp.json()

{'detail': 'Not authenticated'}

<IPython.core.display.Javascript object>

In [5]:
endpoint = "/auth/login"
data = {"username": "user1", "password": "pass"}

resp = client.post(endpoint, data=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [6]:
resp.json()

{'access_token': 'user1', 'token_type': 'bearer'}

<IPython.core.display.Javascript object>

In [7]:
client.headers["Authorization"] = "bearer user1"
resp = client.get("/me")
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [8]:
resp.json()

{'id': '4f09ec85-34c4-4f49-96d3-f5cbe4b6f8f1',
 'created_at': '2021-11-04T20:05:45.952918+00:00',
 'name': 'user1'}

<IPython.core.display.Javascript object>

In [9]:
resp.request.headers

Headers({'host': 'backend:8000', 'accept': '*/*', 'accept-encoding': 'gzip, deflate, br', 'connection': 'keep-alive', 'user-agent': 'python-httpx/0.20.0', 'authorization': '[secure]'})

<IPython.core.display.Javascript object>

# Get todos

In [10]:
resp = client.get("/todo")
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [11]:
resp.json()

[{'id': '02e53fa1-e6a5-4cfd-941a-558b49c66c24',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 2',
  'content': 'Edit this Note'},
 {'id': '951011fc-46d7-4139-a1bb-ec5d69793081',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 4',
  'content': 'Reminder: make more Notes'},
 {'id': 'a96ce9e4-ea44-4c51-aa92-bd424c2f8f42',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'ab3f4002-96ec-4fee-a987-26466e856f8e',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 3',
  'content': 'So many Notes'}]

<IPython.core.display.Javascript object>

In [12]:
# note it followed a redirect!
resp.history

[<Response [307 Temporary Redirect]>]

<IPython.core.display.Javascript object>

In [13]:
resp.history[0].headers

Headers({'date': 'Thu, 04 Nov 2021 20:06:04 GMT', 'server': 'uvicorn', 'location': 'http://backend:8000/todo/', 'transfer-encoding': 'chunked'})

<IPython.core.display.Javascript object>

# Make a todo

In [14]:
data = {"title": "programmatic Note", "content": "made from python"}
resp = client.post("/todo", json=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [15]:
resp.request.read()

b'{"title": "programmatic Note", "content": "made from python"}'

<IPython.core.display.Javascript object>

In [16]:
resp.json()

{'id': 'e9bf2779-30f5-4d96-8ee1-d749ffa0e819',
 'created_at': '2021-11-04T20:06:12.837594+00:00',
 'title': 'programmatic Note',
 'content': 'made from python'}

<IPython.core.display.Javascript object>

In [17]:
# check that it exists in the "get all" now
resp = client.get("/todo")
resp.json()

[{'id': '02e53fa1-e6a5-4cfd-941a-558b49c66c24',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 2',
  'content': 'Edit this Note'},
 {'id': '951011fc-46d7-4139-a1bb-ec5d69793081',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 4',
  'content': 'Reminder: make more Notes'},
 {'id': 'a96ce9e4-ea44-4c51-aa92-bd424c2f8f42',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'ab3f4002-96ec-4fee-a987-26466e856f8e',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 3',
  'content': 'So many Notes'},
 {'id': 'e9bf2779-30f5-4d96-8ee1-d749ffa0e819',
  'created_at': '2021-11-04T20:06:12.837594+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'}]

<IPython.core.display.Javascript object>

# Update a todo

In [18]:
todo = resp.json()[0]
todo

{'id': '02e53fa1-e6a5-4cfd-941a-558b49c66c24',
 'created_at': '2021-11-04T20:05:45.952918+00:00',
 'title': 'Note 2',
 'content': 'Edit this Note'}

<IPython.core.display.Javascript object>

In [19]:
_id = todo["id"]
endpoint = f"/todo/{_id}"

data = {"content": "updated by python"}
resp = client.put(endpoint, json=data)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [20]:
resp.json()

{'id': '02e53fa1-e6a5-4cfd-941a-558b49c66c24',
 'created_at': '2021-11-04T20:05:45.952918+00:00',
 'title': 'Note 2',
 'content': 'updated by python'}

<IPython.core.display.Javascript object>

In [21]:
resp = client.get("/todo")
resp.json()

[{'id': '02e53fa1-e6a5-4cfd-941a-558b49c66c24',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 2',
  'content': 'updated by python'},
 {'id': '951011fc-46d7-4139-a1bb-ec5d69793081',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 4',
  'content': 'Reminder: make more Notes'},
 {'id': 'a96ce9e4-ea44-4c51-aa92-bd424c2f8f42',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'ab3f4002-96ec-4fee-a987-26466e856f8e',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 3',
  'content': 'So many Notes'},
 {'id': 'e9bf2779-30f5-4d96-8ee1-d749ffa0e819',
  'created_at': '2021-11-04T20:06:12.837594+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'}]

<IPython.core.display.Javascript object>

# Delete a todo

In [22]:
resp = client.delete(endpoint)
resp

<Response [200 OK]>

<IPython.core.display.Javascript object>

In [23]:
# calling it again shows 404, already deleted
resp = client.delete(endpoint)
resp

<Response [404 Not Found]>

<IPython.core.display.Javascript object>

In [24]:
resp = client.get("/todo")
resp.json()

[{'id': '951011fc-46d7-4139-a1bb-ec5d69793081',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 4',
  'content': 'Reminder: make more Notes'},
 {'id': 'a96ce9e4-ea44-4c51-aa92-bd424c2f8f42',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 1',
  'content': 'My first Note'},
 {'id': 'ab3f4002-96ec-4fee-a987-26466e856f8e',
  'created_at': '2021-11-04T20:05:45.952918+00:00',
  'title': 'Note 3',
  'content': 'So many Notes'},
 {'id': 'e9bf2779-30f5-4d96-8ee1-d749ffa0e819',
  'created_at': '2021-11-04T20:06:12.837594+00:00',
  'title': 'programmatic Note',
  'content': 'made from python'}]

<IPython.core.display.Javascript object>